In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
#pip install xgboost

%matplotlib inline
sns.set_style("whitegrid")


In [3]:
calendar = pd.read_csv('D:/calendar.csv')
sales_train_validation = pd.read_csv('D:/sales_train_validation.csv')
sell_prices = pd.read_csv('D:/sell_prices.csv')

In [4]:
display(calendar.head())
display(sales_train_validation.head())
display(sell_prices.head())

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [5]:
#Save memory
def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == np.object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df  

In [6]:
calendar = downcast(calendar)
sales_train_validation = downcast(sales_train_validation)
sell_prices = downcast(sell_prices)

C:\Users\qluaj\AppData\Local\Temp/ipykernel_7292/4236345101.py:22: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif t == np.object:


In [7]:
calendar.to_csv('D:/calendar_red.csv')
sales_train_validation.to_csv('D:/sales_train_validation_red.csv')
sell_prices.to_csv('D:/sell_prices_red.csv')

In [8]:
submission=pd.read_csv('D:/sample_submission.csv')

In [9]:
from sklearn import preprocessing, metrics
import gc
import os

In [17]:
def reshape_and_merge(calendar, sell_prices, sales_train_validation, submission, merge=False):
    
    # , nrows=55000000
    sales_train_validation = pd.melt(sales_train_validation, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    print('Reshaped sales_train_validation has {} rows and {} columns'.format(sales_train_validation.shape[0], sales_train_validation.shape[1]))
    
    test_rows = [row for row in submission['id'] if 'validation' in row]
    val_rows = [row for row in submission['id'] if 'evaluation' in row]
    test = submission[submission['id'].isin(test_rows)]
    val = submission[submission['id'].isin(val_rows)]
    
    test.columns = ['id', 'd_1914', 'd_1915', 'd_1916', 'd_1917', 'd_1918', 'd_1919', 'd_1920', 'd_1921', 
                    'd_1922', 'd_1923', 'd_1924', 'd_1925', 'd_1926', 'd_1927', 'd_1928', 'd_1929', 'd_1930', 
                    'd_1931', 'd_1932', 'd_1933', 'd_1934', 'd_1935', 'd_1936', 'd_1937', 'd_1938', 'd_1939', 
                    'd_1940', 'd_1941']
    val.columns = ['id', 'd_1942', 'd_1943', 'd_1944', 'd_1945', 'd_1946', 'd_1947', 'd_1948', 'd_1949', 
                   'd_1950', 'd_1951', 'd_1952', 'd_1953', 'd_1954', 'd_1955', 'd_1956', 'd_1957', 'd_1958', 
                   'd_1959', 'd_1960', 'd_1961', 'd_1962', 'd_1963', 'd_1964', 'd_1965', 'd_1966', 'd_1967', 
                   'd_1968', 'd_1969']
    
    product = sales_train_validation[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']].drop_duplicates()
    
    test = test.merge(product, how = 'left', on = 'id')
    val = val.merge(product, how = 'left', on = 'id')
    
    test = pd.melt(test, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    val = pd.melt(val, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    
    sales_train_validation['part'] = 'train'
    test['part'] = 'test'
    val['part'] = 'val'
    
    data = pd.concat([sales_train_validation, test, val], axis = 0)
    
    del sales_train_validation, test, val
    
    #data = data.loc[nrows:]
    
    data = data[data['part'] != 'val']

    if merge:
        data = pd.merge(data, calendar, how = 'left', left_on = ['day'], right_on = ['d'])
        data.drop(['d', 'day'], inplace = True, axis = 1)
        data = data.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')
        print('The dataset finally has {} rows and {} columns'.format(data.shape[0], data.shape[1]))
    else: 
        pass
    
    return data




In [18]:
data = reshape_and_merge(calendar, sell_prices, sales_train_validation, submission, merge=True)

Reshaped sales_train_validation has 58327370 rows and 8 columns
The dataset finally has 59181090 rows and 22 columns


In [19]:
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN


In [20]:
data.shape

(59181090, 22)

In [54]:
def transform(data):
    
    nan_features = [ 'event_type_1', 'event_name_2', 'event_type_2']
    for feature in nan_features:
        data[feature]=pd.Categorical(data[feature])
        data[feature].cat.add_categories('unknown',inplace=True)
        data[feature].fillna('unknown', inplace = True)
        
    cat = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    for feature in cat:
        encoder = preprocessing.LabelEncoder()
        data[feature] = encoder.fit_transform(data[feature])
    
    return data

In [55]:
data=transform(data)

In [56]:
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,0,train,2011-01-29,11101,...,1,2011,30,4,4,2,0,0,0,NaN
1,HOBBIES_1_002_CA_1_validation,1438,3,1,0,0,0,train,2011-01-29,11101,...,1,2011,30,4,4,2,0,0,0,NaN
2,HOBBIES_1_003_CA_1_validation,1439,3,1,0,0,0,train,2011-01-29,11101,...,1,2011,30,4,4,2,0,0,0,NaN
3,HOBBIES_1_004_CA_1_validation,1440,3,1,0,0,0,train,2011-01-29,11101,...,1,2011,30,4,4,2,0,0,0,NaN
4,HOBBIES_1_005_CA_1_validation,1441,3,1,0,0,0,train,2011-01-29,11101,...,1,2011,30,4,4,2,0,0,0,NaN


In [57]:
def simple_data(data):
    
    # rolling demand features
    data['lag_t28'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28))
    # price features
    data['lag_price_t1'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1))
    data['price_change_t1'] = (data['lag_price_t1'] - data['sell_price']) / (data['lag_price_t1'])
    data['rolling_price_max_t365'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1).rolling(365).max())
    data['price_change_t365'] = (data['rolling_price_max_t365'] - data['sell_price']) / (data['rolling_price_max_t365'])
    data.drop(['rolling_price_max_t365', 'lag_price_t1'], inplace = True, axis = 1)
    # time features
    data['date'] = pd.to_datetime(data['date'])
    data['year'] = data['date'].dt.year
    data['month'] = data['date'].dt.month
    data['week'] = data['date'].dt.isocalendar().week
    data['day'] = data['date'].dt.day
    data['dayofweek'] = data['date'].dt.dayofweek
    
    
    return data

In [58]:
data_simple=simple_data(data)

C:\Users\qluaj\AppData\Local\Temp/ipykernel_7292/4239454545.py:31: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  data['week'] = data['date'].dt.week


In [59]:
data_simple=downcast(data_simple)

C:\Users\qluaj\AppData\Local\Temp/ipykernel_7292/4236345101.py:22: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif t == np.object:
C:\Users\qluaj\AppData\Local\Temp/ipykernel_7292/4236345101.py:22: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif t == np.object:


In [60]:
#Selected features
features = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'year', 'month', 'week', 'day', 'dayofweek', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 
            'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'lag_t28', 'rolling_mean_t7', 
            'price_change_t1', 'price_change_t365']


In [62]:
data_simple.shape

(59181090, 39)

In [65]:
train_X=data_simple[data_simple['date']<= '2016-03-27']
train_y=train_X['demand']
train_X=train_X[features]

validation_X=data_simple[(data_simple['date']> '2016-03-27') & (data_simple['date']<= '2016-04-24')]
validation_y=validation_X['demand']
validation_X=validation_X[features]

test_X=data_simple[data_simple['date']>= '2016-04-25']

del data_simple
gc.collect()

In [67]:
train_X.shape,validation_X.shape,test_X.shape

((57473650, 32), (853720, 32), (853720, 39))

In [68]:
train_y.shape,validation_y.shape

((57473650,), (853720,))

In [71]:
train_X.to_csv('D:/trainX.csv')
validation_X.to_csv('D:/validationX.csv')
test_X.to_csv('D:/testX.csv')
train_y.to_csv('D:/trainy.csv')
validation_y.to_csv('D:/validationy.csv')

In [84]:
#Simple try
train_X_1=train_X.loc[0:1000]
train_y_1=train_y.loc[0:1000]

In [72]:
from sklearn.metrics import mean_absolute_error

In [87]:
model = XGBRegressor()
model.fit(train_X_1,train_y_1)


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=6,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [97]:
predict_xgb=model.predict(validation_X)
validation_score=np.sqrt(mean_absolute_error(predict_xgb,validation_y))
validation_score

1.5569739

In [111]:
test_X=pd.read_csv('D:/traindataset/testX.csv')

In [99]:
testPre_xgb=model.predict(test_X[features])


In [112]:
def submit_table(pred, test, submission):
    test['demand'] = pred
    predictions = test[['id', 'date', 'demand']]
    predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'demand').reset_index()
    predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]
    evaluation_rows = [row for row in submission['id'] if 'evaluation' in row] 
    evaluation = submission[submission['id'].isin(evaluation_rows)]
    validation = submission[['id']].merge(predictions, on = 'id')
    final = pd.concat([validation, evaluation])
    final.to_csv('D:/submission.csv', index = False)

In [113]:
submit_table(testPre_xgb, test_X, submission)

In [120]:
def fit_xgbModel(train_X,train_y,validation_X,validation_y,lr=0.3,ne=100):
    model = XGBRegressor(learning_rate=lr,n_estimators=ne)
    model.fit(train_X,train_y)
    predict_xgb=model.predict(validation_X)
    validation_score=np.sqrt(mean_absolute_error(predict_xgb,validation_y))
    print('The learning rate is {} and the estimator number is {}. The validation score is {}.'.format(lr,ne,validation_score))
    
    return validation_score

In [132]:
trainX_1=train_X.loc[0:20000000]
trainy_1=train_y.loc[0:20000000]
learn_rate=[0.01,0.05,0.1,0.2]
n_est=[5,10,30,50,100]
val_score_list=[]
for l_r in learn_rate:
    for n_e in n_est:
        val_score=fit_xgbModel(trainX_1,trainy_1,validation_X,validation_y,lr=l_r,ne=n_e)
        

The learning rate is 0.01 and the estimator number is 5. The validation score is 1.1898062229156494.
The learning rate is 0.01 and the estimator number is 10. The validation score is 1.1764861345291138.
The learning rate is 0.01 and the estimator number is 30. The validation score is 1.1327344179153442.
The learning rate is 0.01 and the estimator number is 50. The validation score is 1.1017646789550781.
The learning rate is 0.01 and the estimator number is 100. The validation score is 1.0598281621932983.
The learning rate is 0.05 and the estimator number is 5. The validation score is 1.1412193775177002.
The learning rate is 0.05 and the estimator number is 10. The validation score is 1.1005175113677979.
The learning rate is 0.05 and the estimator number is 30. The validation score is 1.0445421934127808.
The learning rate is 0.05 and the estimator number is 50. The validation score is 1.0385396480560303.
The learning rate is 0.05 and the estimator number is 100. The validation score is 